# Method Of Manufactured Solutions for SWIRL
## Introduction

The Method of Manufactured Solutions (MMS) is a process for generating an
analytical solution for a code that provides the numerical solution for a given
domain. The goal of MMS is to establish a manufactured solution that can
be used to establish the accuracy of the code within question. For this study,
SWIRL, a code used to calculate the radial modes within an infinitely long
duct is being validated through code verification. SWIRL accepts a given
mean flow and uses numerical integration to obtain the speed of sound. The
integration technique is found to be the composite trapezoidal rule through
asymptotic error analysis.

## Methods 
The SWIRL code requires two mean flow parameters as a function of radius
e is calculated by integrating M θ
dius, M x , and M θ . The speed of sound, A
with respect to r. To verify that SWIRL is handling and returning the ac-
companying mean flow parameters, the error between the mean flow input
and output variables are computed. Since the trapezoidal rule is used to
numerically integrate M θ , the discretization error and order of accuracy is
computed. Since finite differencing schemes are to be used on the result of
this integration, it is crucial to accompany the integration with methods of
equal or less order of accuracy. This will be determined by applying another
MMS on the eigenproblem which will also have an order of accuracy.



In [1]:
# Set Up Libraries
%run 1-Set-Up.ipynb


In [2]:
# %load ../../Documentation/meanFlowMMS/introduction.tex


In [3]:
# Defining symbolic variables needed for this code.
# all units are dimensionless! 

# radius, maximum radius, and ratio of specific heats
r, r_max, kappa = symbols('r r_max kappa')

# arbitrary constants
k = symbols('k', cls=IndexedBase)
one, two, three = symbols('one two three')
C = symbols('C')
x = symbols('x')

# reduced frequency
ak = symbols('ak')

# imaginary #, Speed of sound, azimuthal mode number, axial wavenumber 
i, A, m, gamma   = symbols('i A m gamma')
v_r, v_t, v_x, p = symbols('v_r v_t v_x p ')
M_t, M_x         = symbols('M_t M_x')
dp_dr, dv_r_dr   = symbols('dp_dr dv_r_dr')
dM_x_dr, dM_t_dr = symbols('dM_x_dr dM_t_dr')

#locations of inflection Points for tanh?
r2 , r3  = symbols('r2 r3')
j        = symbols('j')

### Theory
#### Speed of Sound Integration from Mean Flow


The goal was to use a function that would be harder to resolve with second
order schemes. A FORTRAN code was written from this notebook to create a              
summation of  $\tanh$  functions. The issue is implementing this with the       
current MMS workflow in SWIRL because of the symbolic expressions that would be needed for the source terms in the  Ax-λ Bx = S. For now we explicitly define 3 kink-anti-kink functions                                    


\begin{align*}                                                                                           
    \bar{A} = \sum_{j=1}^n R_{ij} + \sum_{j=1}^n L_{ij}                                                  
\end{align*}                                                                                             
where,                                                                                                   
\begin{align*}                                                                                           
    R_{ij} = A_j \tanh(B_j (x_i - x_j)) \\                                                               
    L_{ij} = A_j \tanh(B_j (x_j - x_n))                                                                  
\end{align*}                                                                                             
Letting $n = 3$

\begin{align*}                                                                                           
    \bar{A} &= S_{vert} + \sum_{j=1}^3 R_{ij} + \sum_{j=1}^3 L_{ij} \\                                   
    \bar{A} &=                                                                                           
    A_1 \tanh(B_1 (x_i - x_1))  +                                                                        
     A_{2} \tanh(B_{2} (x_i - x_{2}))  +                                                                 
     A_{3} \tanh(B_{3} (x_i - x_3)) +  \\                                                                
    A_1 \tanh(B_1 (x_1 - x_n)) &+                                                                        
     A_{2} \tanh(B_{2} (x_2 - x_{n}))  +                                                                 
     A_{3} \tanh(B_{3} (x_3 - x_n))                                                                      
\end{align*}                                                                                             
and,                                                                                                     
\begin{align*}                                                                                           
   A_1 = A_2 = A_3 = k_1 \\                                                                              
   B_1 = B_2 = B_3 = k_2                                                                                 
\end{align*}                                                                                                                                                                                             
                      

In [4]:
RightKink = \
k[1]*sp.tanh(k[2]*(r - r_max)) + \
k[1]*sp.tanh(k[2]*(r - r2)) + \
k[1]*sp.tanh(k[2]*(r - r3))

In [5]:
LeftKink = \
k[1]*sp.tanh(k[2]*(r_max - r_max)) + \
k[1]*sp.tanh(k[2]*(r2    - r_max)) + \
k[1]*sp.tanh(k[2]*(r3    - r_max))

#RightKink = Sum(sp.tanh(r - Indexed('r_loc',j)),(j,0,2))
#LeftKink = Sum(sp.tanh(Indexed('r_loc',j) - r_max),(j,0,2))
# had a hard time converting this to fortran! >:[

A_analytic =  one + LeftKink + RightKink 

dA_analytic_dr    = diff(A_analytic,r)
dA_analytic_sq_dr = diff(A_analytic**two,r)



M_t_analytic = sp.sqrt( \
                       r/((kappa-one)*A_analytic**two) * \
                       (dA_analytic_sq_dr))


#### Eigenvalue Decomposition
The SWIRL code performs an eigenvalue decomposition of this set of equations:

$$ [A]{x} = \lambda [B] {x} $$

which can be rearranged as,

$$ [A]{x} - \lambda [B] {x} = 0$$

Here, $x$ is an eigenvector composed of the perturbation variables, $v_r,v_{\theta},v_x,p$ and $\lambda$ is the associated eigenvalue, (Note: $\lambda = -i \bar{\gamma}$)


Writing this out we obtain ....

Linear System of Equations:


(insert source terms here)
Defining perturbation variables

In [6]:
M_x_analytical = k[3]*sp.cos(k[3]*(r - r_max))
v_r_analytical = k[4]*sp.cos(k[4]*(r - r_max))
v_t_analytical = k[5]*sp.cos(k[5]*(r - r_max))
v_x_analytical = k[6]*sp.cos(k[6]*(r - r_max))
p_analytical   = k[7]*sp.cos(k[7]*(r - r_max))

In [7]:
S_1 = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*v_r  -\
\
(two/r)*M_t*v_t + dp_dr*(1/p) + ((kappa - one)/r)*(M_t**two)*p

S_2 = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*v_t + \
\
(M_t/r + dM_t_dr + ((kappa - one)/(two*r))*M_t**three)*v_r + i*m*p/r

S_3 = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*v_x + \
\
(dM_x_dr + ((kappa - one)/(two*r))*M_x*M_t**two)*v_r + i*one*gamma*p
 
S_4 = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*p   \
\
+ dv_r_dr + (((kappa + one)/(two*r))*M_t**two + one/r)*v_r + i*m*v_t/r + i*one*gamma*v_x

# Lets look at the source terms
pprint('The linearized (unsteady) Euler Equations used in SWIRL:')
pprint(('S_1=',S_1))
pprint(('S_2=',S_2))
pprint(('S_3=',S_3))
pprint(('S_4=',S_4))

The linearized (unsteady) Euler Equations used in SWIRL:
⎛                      two                                                ⎞
⎜        Mₜ⋅two⋅vₜ   Mₜ   ⋅p⋅(κ - one)   dp_dr        ⎛  Mₜ⋅m          ak⎞⎟
⎜S_1=, - ───────── + ───────────────── + ───── - i⋅vᵣ⋅⎜- ──── - Mₓ⋅γ + ──⎟⎟
⎝            r               r             p          ⎝   r            A ⎠⎠
⎛                                             ⎛       three                   
⎜      i⋅m⋅p        ⎛  Mₜ⋅m          ak⎞      ⎜Mₜ   Mₜ     ⋅(κ - one)         
⎜S_2=, ───── - i⋅vₜ⋅⎜- ──── - Mₓ⋅γ + ──⎟ + vᵣ⋅⎜── + ───────────────── + dM_t_d
⎝        r          ⎝   r            A ⎠      ⎝r          r⋅two               

 ⎞⎞
 ⎟⎟
r⎟⎟
 ⎠⎠
⎛                                                 ⎛  two                      
⎜                       ⎛  Mₜ⋅m          ak⎞      ⎜Mₜ   ⋅Mₓ⋅(κ - one)         
⎜S_3=, γ⋅i⋅one⋅p - i⋅vₓ⋅⎜- ──── - Mₓ⋅γ + ──⎟ + vᵣ⋅⎜────────────────── + dM_x_d
⎝                       ⎝   r            A ⎠      ⎝      r⋅two       

In [8]:
AA = sp.Matrix(sp.zeros(4,4))

AA[0,0] = -i*(ak/A - (m/r)*M_t)
AA[1,1] = AA[0,0]
AA[2,2] = AA[0,0]
AA[3,3] = AA[0,0]

AA[0,2] = 0
AA[1,2] = 0
AA[3,2] = 0
AA[2,1] = 0
AA[2,3] = 0

AA[1,3] = (i*m)/r
AA[3,1] = AA[1,3]

A12 = (-two/r)*M_t
A21 = M_t/r + dM_t_dr + (kappa - one)/(two*r)*M_t**three
A31 = dM_x_dr + (kappa - one)/(two*r)*M_t**two*M_x

AA[0,1] = A12
AA[1,0] = A21
AA[2,0] = A31

# note that these terms only have derivative operators , See Eqn 2.52

A41 = (1/v_r)*dv_r_dr + one/r + (kappa + one)/(two*r)*M_t**two
A14 = (1/p)*dp_dr + (kappa - one)/r * M_t**two

AA[3,0] = A41
AA[0,3] = A14

BB = sp.zeros(4,4)

BB[0,0] = M_x
BB[1,1] = BB[0,0]
BB[2,2] = BB[0,0]
BB[3,3] = BB[0,0]

BB[2,3] = one
BB[3,2] = BB[2,3]

XX = sp.Matrix([\
               [v_r] , \
               [v_t] , \
               [v_x] , \
               [p] , \
               ])

Lambda = -i*gamma

SS = AA*XX - Lambda*BB*XX

print(S_1.equals(SS[0]))
print(S_2.equals(SS[1]))
print(S_3.equals(SS[2]))
print(S_4.equals(SS[3]))

print(S_1)
print(SS[0]) 
# multiplying (1/p) and not (one/p) by dp_dr made  S_1 and SS[1] the same

print(S_2)
print(SS[1])
print(S_3)
print(SS[2]) # multiplied i gamma p term by "one" in S_3

pprint(S_4)
pprint(SS[3]) # multiplied by 1/v_r by d_v_dr and i v_x gamma terM by one


False
True
True
True
-M_t*two*v_t/r + M_t**two*p*(kappa - one)/r + dp_dr/p - i*v_r*(-M_t*m/r - M_x*gamma + ak/A)
-M_t*two*v_t/r + M_x*gamma*i*v_r - i*v_r*(-M_t*m/r + ak/A) + p*(M_t**two*(kappa - one)/r + dp_dr/p)
i*m*p/r - i*v_t*(-M_t*m/r - M_x*gamma + ak/A) + v_r*(M_t/r + M_t**three*(kappa - one)/(r*two) + dM_t_dr)
M_x*gamma*i*v_t + i*m*p/r - i*v_t*(-M_t*m/r + ak/A) + v_r*(M_t/r + M_t**three*(kappa - one)/(r*two) + dM_t_dr)
gamma*i*one*p - i*v_x*(-M_t*m/r - M_x*gamma + ak/A) + v_r*(M_t**two*M_x*(kappa - one)/(r*two) + dM_x_dr)
M_x*gamma*i*v_x + gamma*i*one*p - i*v_x*(-M_t*m/r + ak/A) + v_r*(M_t**two*M_x*(kappa - one)/(r*two) + dM_x_dr)
                                                              ⎛  two          
                       i⋅m⋅vₜ       ⎛  Mₜ⋅m          ak⎞      ⎜Mₜ   ⋅(κ + one)
dv_r_dr + γ⋅i⋅one⋅vₓ + ────── - i⋅p⋅⎜- ──── - Mₓ⋅γ + ──⎟ + vᵣ⋅⎜───────────────
                         r          ⎝   r            A ⎠      ⎝     r⋅two     

      ⎞
   one⎟
 + ───⎟
    r ⎠
      

In [9]:
S_1 = (S_1.subs({A:A_analytic, M_t:M_t_analytic}))
SS_1 = (SS[0].subs({A:A_analytic, M_t:M_t_analytic}))
S_2 = (S_2.subs({A:A_analytic, M_t:M_t_analytic}))
SS_2 = (SS[1].subs({A:A_analytic, M_t:M_t_analytic}))
S_3 = (S_3.subs({A:A_analytic, M_t:M_t_analytic}))
S_4 = (S_4.subs({A:A_analytic, M_t:M_t_analytic}))
#pprint(S_1)
#pprint(SS_1)
print(S_1.equals(SS_1))
print(S_2.equals(SS_2))


False
True


In [10]:



S_1 = (S_1.subs({M_x:M_x_analytical, \
                 v_r:v_r_analytical, \
                 v_t:v_t_analytical, \
                 v_x:v_x_analytical, \
                 p:p_analytical, \
                }))

S_2 = (S_2.subs({M_x:M_x_analytical, \
                 v_r:v_r_analytical, \
                 v_t:v_t_analytical, \
                 v_x:v_x_analytical, \
                 p:p_analytical, \
                }))

S_3 = (S_3.subs({M_x:M_x_analytical, \
                 v_r:v_r_analytical, \
                 v_t:v_t_analytical, \
                 v_x:v_x_analytical, \
                 p:p_analytical, \
                }))

S_4 = (S_4.subs({M_x:M_x_analytical, \
                 v_r:v_r_analytical, \
                 v_t:v_t_analytical, \
                 v_x:v_x_analytical, \
                 p:p_analytical, \
                }))

In [11]:
dp_dr_analytical   = p_analytical.diff(r)
dv_r_dr_analytical = v_r_analytical.diff(r) 
dM_x_dr_analytical = M_x_analytical.diff(r)
dM_t_dr_analytical = M_t_analytic.diff(r)

S_1 = (S_1.subs({ \
                 dp_dr:dp_dr_analytical, \
                 dv_r_dr:dv_r_dr_analytical, \
                 dM_x_dr:dM_x_dr_analytical, \
                 dM_t_dr:dM_t_dr_analytical, \
                }))

S_2 = (S_2.subs({ \
                 dp_dr:dp_dr_analytical, \
                 dv_r_dr:dv_r_dr_analytical, \
                 dM_x_dr:dM_x_dr_analytical, \
                 dM_t_dr:dM_t_dr_analytical, \
                })) 

S_3 = (S_3.subs({ \
                 dp_dr:dp_dr_analytical, \
                 dv_r_dr:dv_r_dr_analytical, \
                 dM_x_dr:dM_x_dr_analytical, \
                 dM_t_dr:dM_t_dr_analytical, \
                }))

S_4 = (S_4.subs({ \
                 dp_dr:dp_dr_analytical, \
                 dv_r_dr:dv_r_dr_analytical, \
                 dM_x_dr:dM_x_dr_analytical, \
                 dM_t_dr:dM_t_dr_analytical, \
                }))


In [12]:
%run create-fortran-file-sound-speed.ipynb
%run create-fortran-file-LEE-source-terms.ipynb

In [13]:
%run create-fortran-file-perturbation-variables.ipynb